# Seletores de Exemplos (Examples Selectors)

Imagine que você possua diversos exemplos para passar como contexto para o seu prompt. Neste caso, você não poderia
passar todos os exemplos, pois isso poderia confundir o modelo. O ideal seria passar apenas os exemplos
mais relevantes para o contexto que você deseja. Para isso, você pode utilizar os seletores de exemplos.

Porém, uma pergunta surge nisso tudo: como você pode selecionar os exemplos mais relevantes para o seu contexto?

É isso que iremos ver aqui.

In [7]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

gemini_api_key = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=gemini_api_key, temperature=0)

/home/cayena/.cache/pypoetry/virtualenvs/langchain-examples-NuBMPSch-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Seletor por tamanho de texto (select by length)

In [5]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

exemplos = [
    {"input": "feliz", "output": "triste"},
    {"input": "grande", "output": "pequeno"},
    {"input": "energético", "output": "Letárgico"},
    {"input": "ensolarado", "output": "nublado"},
    {"input": "doce", "output": "salgado"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    examples=exemplos,
    example_prompt=example_prompt,
    max_length=25
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Dê o antônimo da entrada do usuário e se baseie nos exemplos abaixo:",
    suffix="Input: {entrada}\nOutput:",
    input_variables=["entrada"],
)

In [6]:
print(dynamic_prompt.format(entrada="Chato"))

Dê o antônimo da entrada do usuário e se baseie nos exemplos abaixo:

Input: feliz
Output: triste

Input: grande
Output: pequeno

Input: energético
Output: Letárgico

Input: ensolarado
Output: nublado

Input: doce
Output: salgado

Input: Chato
Output:


In [8]:
resp_llm = llm.invoke(dynamic_prompt.format(entrada="Chato"))

In [10]:
print(resp_llm.content)

Interessante


## Seletor baseado em Máxima Relevância Marginal (select by Maximum Marginal Relevance [MRR])

Aqui esse seletor se baseia em distância de cosseno de Embeddings. Ele seleciona exemplos que são mais relevantes para o contexto

In [19]:
# Embeddings Open Source e em Português.
from langchain_community.embeddings import HuggingFaceEmbeddings

modelPath = "intfloat/multilingual-e5-small"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs 
)

In [20]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_community.vectorstores import FAISS

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

In [21]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=exemplos,
    embeddings=embeddings,
    vectorstore_cls=FAISS,
    k=2,
)
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Dê o antônimo da entrada do usuário e se baseie nos exemplos abaixo:",
    suffix="Input: {entrada}\nOutput:",
    input_variables=["entrada"],
)

In [27]:
print(mmr_prompt.format(entrada="Alto"))

Dê o antônimo da entrada do usuário e se baseie nos exemplos abaixo:

Input: grande
Output: pequeno

Input: doce
Output: salgado

Input: Alto
Output:


In [33]:
# Testando com o SemanticSimilaritySelector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=exemplos,
    embeddings=embeddings,
    vectorstore_cls=FAISS,
    k=1,
)
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Dê o antônimo da entrada do usuário e se baseie nos exemplos abaixo:",
    suffix="Input: {entrada}\nOutput:",
    input_variables=["entrada"],
)

In [34]:
print(mmr_prompt.format(entrada="Alto"))

Dê o antônimo da entrada do usuário e se baseie nos exemplos abaixo:

Input: grande
Output: pequeno

Input: Alto
Output:


In [35]:
# Selecionando os mais próximos via embeddings:
selected_examples = example_selector.select_examples({"entrada": "Alto"})
print(selected_examples)

[{'input': 'grande', 'output': 'pequeno'}]
